In [2]:
import numpy as np
import pandas as pd
import csv
import nltk
import re


In [3]:
# read in description txt (5 sentences each)
def read_des_train():
    train_path = './data/descriptions_train/' 
    des_train = []
    for i in range(10000):
        file_name = str(i) + '.txt'
        file_path = train_path + file_name
        des = []
        with open(file_path) as f:
            for line in f.readlines():
                des.append(line.strip('\n'))
        des_train.append(des)
    return des_train


In [4]:
desc_train = read_des_train()

In [5]:
# read in tags
def read_tag_train():
    train_path = './data/tags_train/' 
    tag_train = []
    cat_list = []
    for i in range(10000):
        file_name = str(i) + '.txt'
        file_path = train_path + file_name
        tag = ''
        with open(file_path) as f:
            for line in f.readlines():
                sup_cat, sub_cat = line.strip('\n').split(':')
                tag = tag + ' ' + sub_cat
                cat_list.append(sub_cat)
        tag.strip()
        tag_train.append(tag)
    return tag_train, cat_list

In [6]:
tags_train, category_list = read_tag_train()
category_list = list(set(category_list))

In [7]:
#print tags_train   # empty space in front of each? 
#print category_list
print len(category_list)
print tags_train
    

80
[' airplane bench skateboard person truck backpack handbag dining table', ' bowl carrot spoon', ' car truck traffic light person', ' person bench frisbee car', ' person baseball bat', ' bed chair mouse keyboard book cup tv laptop', ' person donut bicycle', ' person tie', ' car person', ' car bus backpack', ' suitcase', ' person baseball bat baseball glove', '', ' bench baseball bat baseball glove person', ' car stop sign', ' wine glass person bottle chair', ' clock person backpack car truck', ' person snowboard', ' dining table person cup cake', ' train potted plant', ' tv cat', ' clock car', ' giraffe', ' person snowboard', ' car traffic light motorcycle truck', ' bird motorcycle', ' dog bicycle person', ' person surfboard', ' tv laptop book', ' bicycle', ' sports ball fire hydrant person', ' teddy bear', ' cell phone person bottle', ' dog', ' sports ball person tennis racket', ' bowl sandwich', ' car motorcycle person chair dining table', ' refrigerator', ' clock', ' dining table 

In [7]:
# bag of words
def process_BoW(desc_data):
    stopwords = nltk.corpus.stopwords.words('english')
    stemmer = nltk.SnowballStemmer("english")
    desc_words_bow = []
    for descs in desc_data:
        temp = ''
        for i in range(len(descs)):
            word_lower = descs[i].lower()
            word_list = re.findall(r"[a-z]+", word_lower)
            word_filter = [stemmer.stem(word) for word in word_list if word not in stopwords]
            for j in range (len(word_filter)):
                temp = temp + ' '+word_filter[j] 
        desc_words_bow.append(temp)
    return desc_words_bow

In [8]:
#dict_bow = process_BoW(desc_train)
desc_word_bow = process_BoW(desc_train)

In [9]:
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer

cv = CountVectorizer(min_df = 15) # frequency threshold
x_bow_train = cv.fit_transform(desc_word_bow)
words_extract = np.array(cv.get_feature_names())
transformer = TfidfTransformer()
x_tf_train = transformer.fit_transform(x_bow_train).toarray()


In [10]:
print len(words_extract)
print type(words_extract)

1224
<type 'numpy.ndarray'>


In [11]:
x_tf_train = pd.DataFrame(x_tf_train, columns = words_extract)


In [12]:
cv = CountVectorizer(vocabulary = category_list)
tags_train_c = cv.fit_transform(tags_train).toarray()

In [13]:
def read_des_test():
    train_path = './data/descriptions_test/' 
    des_test = []
    for i in range(2000):
        file_name = str(i) + '.txt'
        file_path = train_path + file_name
        des = []
        with open(file_path) as f:
            for line in f.readlines():
                des.append(line.strip('\n'))
        des_test.append(des)
    return des_test

desc_test = read_des_test()

In [14]:
desc_test_bow = process_BoW(desc_test) # To do: process BOW 

In [15]:
# process test data


cv = CountVectorizer(vocabulary = words_extract)
x_bow_test = cv.fit_transform(desc_test_bow)
transformer = TfidfTransformer()
x_test_tf = transformer.fit_transform(x_bow_test).toarray()

# Convert into DataFrames
x_test_tf = pd.DataFrame(x_test_tf, columns = words_extract)



In [ ]:
print len(x_tf_train)

In [28]:
# PCA 
from sklearn.decomposition import PCA

pca_pool5 = PCA(n_components=1224, svd_solver='auto') # 2048 -> change threshold 
pca_pool5.fit(x_tf_train)
bow_all_train_PCA = pca_pool5.transform(x_tf_train)
#bow_all_test_PCA = pca_all_pool5.transform(bow_all_test)

In [29]:
pool5_train_data = pd.read_csv(filepath_or_buffer="./data/features_train/features_resnet1000intermediate_train.csv",header=None
)
pool5_train_data[0] = pool5_train_data[0].apply(lambda x: int(x.split('/')[1].replace('.jpg','')))
pool5_train_sort = pool5_train_data.sort_values(by=[0])
pool5_train_images = pool5_train_sort[0].values
pool5_train = pool5_train_sort[list(range(1,2049))].values

In [ ]:
from sklearn.cross_decomposition import PLSRegression

pls_all_pool5 = PLSRegression(n_components=1224)
pls_all_pool5.fit(bow_all_train_PCA, pool5_train)
#pickle.dump(pls_all_pool5, open(ROOT+"models/"+'pls_all_pool5_2048c.sav', 'wb'))


In [48]:
# Ramdom Forest

from sklearn.ensemble import RandomForestRegressor
from sklearn import preprocessing

predict_tags = []
for i in range(80):
    print i
    y = tags_train_c[:,i]
    clf = RandomForestRegressor()
    clf.fit(x_tf_train, y)
    tag = clf.predict(x_test_tf)
    predict_tags.append(tag)
    

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79


In [ ]:
#from sklearn import preprocessing
#train_p = preprocessing.normalize(train_feature, norm='l1')

In [49]:
print len(predict_tags)

80


In [50]:
predict_tags = np.array(predict_tags)
predict_tags_t = predict_tags.T


In [51]:
def read_tag_test():
    train_path = './data/tags_test/' 
    tag_test = []
    for i in range(2000):
        file_name = str(i) + '.txt'
        file_path = train_path + file_name
        tag = ''
        with open(file_path) as f:
            for line in f.readlines():
                sup_cat, sub_cat = line.strip('\n').split(':')
                tag = tag + ' ' + sub_cat
        tag.strip()
        tag_test.append(tag)
    return tag_test #, cat_list

tags_test = read_tag_test()

In [52]:
cv = CountVectorizer(vocabulary = category_list)
tags_test_c = cv.fit_transform(tags_test)


In [53]:
# knn
from sklearn.neighbors import NearestNeighbors as KNN

label_test = []
for i in range(2000):
    label_test.append(i)
knn = KNN(n_neighbors = 20)
knn = knn.fit(tags_test_c, label_test)
prediction = knn.kneighbors(predict_tags_t, return_distance = False)

In [54]:
img_list = []
for row in (prediction):
    img_num = ''
    for val in row:
        img_num = img_num + ' ' + (str(val) + ".jpg")
    img_list.append(img_num)
    
images = pd.DataFrame(img_list, columns = ["Top_20_Image_IDs"])

In [55]:
idx = []
for i in range(2000):
    idx.append(str(i)+ ".txt")
idx = pd.DataFrame(idx, columns = ["Descritpion_ID"])


In [56]:
result = pd.concat([idx, images], axis=1)
result.to_csv("test2_result.csv", index = False)